# Neutron Powder Diffraction

In this tutorial demonstrates how neutron-scattering data can be loaded, visualized, and manipulated with generic functionality from `scipp` as well as neutron-specific functionality from `scippneutron`. It focuses on reducing data from the ORNL [POWGEN](https://neutrons.ornl.gov/powgen) neutron diffractometer.

In [ ]:
import numpy as np
import scipp as sc
import scippneutron as scn

### Loading Nexus files

Loading Nexus files requires [Mantid](https://www.mantidproject.org).
See, e.g., [Installation](https://scipp.github.io/getting-started/installation.html) on how to install scipp and Mantid with `conda`.

We start by loading two files: the sample and the vanadium runs.

In [ ]:
raw_sample = scn.load_with_mantid(
    scn.data.get_path('PG3_4844_event.nxs'),
    load_pulse_times=False,
    mantid_args={'LoadMonitors': True},
)
raw_vanadium = scn.load_with_mantid(
    scn.data.get_path('PG3_4866_event.nxs'), load_pulse_times=False
)

The optional `mantid_args` dict is forwarded to the Mantid algorithm used for loading the files &ndash; in this case [LoadEventNexus](https://docs.mantidproject.org/nightly/algorithms/LoadEventNexus-v1.html) &ndash; and can be used to control, e.g., which part of a file to load.
Here we request loading monitors, which Mantid does not load by default.
The resulting dataset looks as follows:

In [ ]:
raw_sample

Extract the actual events.

In [ ]:
sample = raw_sample['data']
vanadium = raw_vanadium['data']

In [ ]:
sc.plot(sample.hist(spectrum=500, tof=400))

### Instrument view

Scipp provides a simple 3D instrument view inspired by Mantid's own [instrument view](https://www.mantidproject.org/MantidPlot:_Instrument_View), which can be used to take a quick look at the neutron counts on the detector panels in 3D space or using various cylindrical and spherical projections

<div class="alert alert-info">

**Note**

The support and handling of event data is currently being rewritten.
The interface for manipulating event data (binned) data is not representative for the final API.

</div>

In [ ]:
scn.instrument_view(sample.hist())

### Plot against scattering angle $\theta$ using `bin`

*This is not an essential step and can be skipped.*

Plotting raw data directly yields a hard-to-interpret figure.
We can obtain something more useful by binning the spectrum axis based on its $\theta$ value:

In [ ]:
sample.coords['two_theta'] = scn.two_theta(sample)
vanadium.coords['two_theta'] = scn.two_theta(vanadium)
two_theta = sc.linspace(dim='two_theta', unit='rad', start=0.0, stop=np.pi, num=2001)

We concatenate events lists from different spectra that fall into a given $2\theta$ range into longer combined lists:

In [ ]:
theta_sample = sample.bin(two_theta=two_theta)

In [ ]:
theta_sample

In [ ]:
sc.plot(theta_sample.hist(tof=400))

### Coordinate transformation

*Note: We are back to working with `sample`, not `theta_sample`.*

`scippneutron` provides building blocks for [scipp.transform_coords](https://scipp.github.io/user-guide/coordinate-transformations.html) to convert between coordinates related to time-of-flight.
The loaded raw data has dimension `tof`, and we convert to interplanar lattice spacing (dspacing):

In [ ]:
dspacing_graph = {
    **scn.conversion.graph.beamline.beamline(scatter=True),
    **scn.conversion.graph.tof.elastic_dspacing('tof'),
}

In [ ]:
dspacing_vanadium = vanadium.transform_coords('dspacing', graph=dspacing_graph)
dspacing_sample = sample.transform_coords('dspacing', graph=dspacing_graph)
dspacing_sample

### Neutron monitors

*This is an optional section.
The next section does not use the monitor-normalized data produced here.
This section could thus be skipped.*

If available, neutron monitors are stored as attributes of a data array:

In [ ]:
mon = raw_sample['monitors']['monitor1']['data']
mon

The monitor could, e.g., be used to normalize the data.
To do so, both data and monitor need to be converted to a unit that accounts for differing flight paths, e.g., wavelength or energy:

In [ ]:
wavelength_graph = {
    **scn.conversion.graph.beamline.beamline(scatter=True),
    **scn.conversion.graph.tof.elastic_wavelength('tof'),
}
wavelength_graph_monitor = {
    **scn.conversion.graph.beamline.beamline(scatter=False),
    **scn.conversion.graph.tof.elastic_wavelength('tof'),
}

In [ ]:
sample_lambda = sample.transform_coords('wavelength', graph=wavelength_graph)
mon = mon.transform_coords('wavelength', graph=wavelength_graph_monitor)

The sample data is in event-mode, i.e., is not histogrammed.
Event data *can* be divided by a histogram (such as `mon` in this case), using a specialized function for scaling (see [Binned data](https://scipp.github.io/user-guide/binned-data.html)).
First we rebin the monitor since the original binning is very fine:

In [ ]:
edges = sc.linspace(dim='wavelength', unit='Angstrom', start=0.0, stop=1.0, num=1001)
mon = sc.rebin(mon, wavelength=edges)
mon

Normalizing by (that is dividing by) the monitor would introduce correlations between different detector pixels if the monitor has variances.
This is because the same monitor bin is applied to many detector bins.
Scipp would reject such an operation.
To work around this, we drop the variances of the monitor.
In practice, we have to carefully examine the uncertainties to find out if they really can be neglected!

In [ ]:
mon = sc.values(mon)

We intend to normalize each event to the relative monitor counts (compared to the total monitor counts).
We use `sum` to compute the total monitor counts and obtain the relative counts using division:

In [ ]:
mon /= mon.sum('wavelength')

The sample data is *event data in bins* and the monitor is a histogram.
Multiplication and division operations for such cases are supported by modifying the weights (values) for each event using the operators of the `bins` property, in combination with the `sc.lookup` helper, a wrapper for a discrete "function", given by the monitor:

In [ ]:
sample_over_mon = sample_lambda.bins / sc.lookup(func=mon, dim='wavelength')
sample_over_mon

Finally, we can plot the data, which needs to be histogrammed before being plotted.
By default, `.hist()` uses the coordinates of the binned data to define histogram edges,
which, in this case, would give a single bin along the `'wavelength'` dimension.
For a better representation of the data, we supply a finer binning, yielding a more meaningful figure:

In [ ]:
sc.plot(sample_over_mon.hist(wavelength=400).hist(spectrum=500).transpose())

In [ ]:
del sample_lambda
del sample_over_mon
del sample
del vanadium

### From events to histogram

*Note: We are continuing here with data that has not been normalized to the monitors.*

We histogram the event data:

In [ ]:
dspacing = sc.arange(dim='dspacing', start=0.3, stop=2.0, step=0.001, unit='Angstrom')
hist = sc.Dataset(
    data={
        'sample': dspacing_sample.hist(dspacing=dspacing),
        'vanadium': dspacing_vanadium.hist(dspacing=dspacing),
    }
)
sc.show(hist['spectrum', 0:3]['dspacing', 0:7])

In [ ]:
sc.plot(hist['sample'].hist(spectrum=500).transpose())

### Summing (focussing) and normalizing

After conversion to `'dspacing'`, generic `sum` and `/` operations can be used to "focus" and normalize the diffraction data to the vanadium run:

In [ ]:
summed = sc.sum(hist, 'spectrum')
sc.plot(summed)

In [ ]:
normalized = summed['sample'] / summed['vanadium']
sc.plot(normalized)

### Focussing with $\theta$ dependence in event-mode

The approach used above combines reflections from all crystallographic planes and is therefore of limited use.
We can use `bin` to focus each of multiple groups of spectra into a distinct output spectrum.
Here we define groups based on a range of scattering angles &ndash; a simple $\theta$-dependent binning.
This also demonstrates how we can postpone histogramming until after the focussing step.

In [ ]:
two_theta = sc.linspace(dim='two_theta', unit='rad', start=0.0, stop=np.pi, num=16)

focussed_sample = dspacing_sample.bin(two_theta=two_theta)
focussed_vanadium = dspacing_vanadium.bin(two_theta=two_theta)
norm = focussed_vanadium.hist(dspacing=dspacing)

Similarly as when normalizing by monitor, we have to drop the variances of the normalization term.
Otherwise, the normalization would be broadcast to `focussed_sample` which would introduce correlations.
In practice, we have to make sure that the uncertainties of the vanadium measurement can be neglected!

In [ ]:
norm = sc.values(norm)

In [ ]:
focussed_sample.bins /= sc.lookup(func=norm, dim='dspacing')
normalized = focussed_sample.hist(dspacing=dspacing)

The normalized output looks as follows:

In [ ]:
normalized.plot(vmin=sc.scalar(0), vmax=sc.scalar(0.4))

As a bonus, we can use slicing and a dict-comprehension to quickly create of plot comparing the spectra for different scattering angle bins:

In [ ]:
# compute centers of theta bins
angle = normalized.coords['two_theta'].values
angle = 0.5 * (angle[1:] + angle[:-1])
results = {
    f'{round(angle[group], 3)} rad': normalized['dspacing', 300:500]['two_theta', group]
    for group in range(2, 6)
}
sc.plot(results)